# Šķēpmešana

#### Inicializācija un bibliotēku imports

In [17]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams.update({'font.size': 20})

#### Gravitācijas konstante

In [18]:
g=9.81


## Teorijas apraksts

Izsviesta šķēpa pārvietošanos raksturojam ar diferenciālvienādojumiem. Pielietots punktveida objekta tuvinājums, gaisa berze izteikta proporcionāla ātruma kvadrātam.
$$\begin{array}{lcl} 
\frac{dx}{dt} &=& v_x \\
\frac{dz}{dt} &=& v_z \\
\frac{dv_x}{dt} &=& -\gamma v_x \sqrt{v_x^2+v_z^2} \\
\frac{dv_z}{dt} &=& -g -\gamma v_z \sqrt{v_x^2+v_z^2}
\end{array}$$
Šeit $x$ un $z$ ir pārvietojums pa, attiecīgi, x un z asi, $v_x$ un $v_z$ ir ātruma komponente pa šīm asīm, $g$ ir brīvās krišānas paātrinājums, $\gamma$ ir gaisa berzes koeficients. Berzes koeficientu izteiksim atkarīgu no *empīriska* stacionārā krišanas ātruma (*terminal velocity*): $\gamma=g/v_{term}^2$.  
Sākumnosacījumi diferenciālvienādojumu sistēmai ir 
$$\begin{array}{lcl} 
x(0) &=& 0 \\
z(0) &=& h_0 \\
v_x(0) &=& v_0 cos \alpha \\
v_z(0) &=& v_0 sin \alpha
\end{array}$$, šeit $h_0$ ir sākotnējais augstums (~2 m), $v_0$ ir izsviešanas ātrums, $\alpha$ ir izsviešanas leņkis pret horizontu.  
Aprēķina uzdevums ir noteikt lidojuma attālumu $x_{att}$. To nosaka, atrodot laika momentu $t_{att}$, kurā $z=0$, t.i. šķēps piezemējas. Aprēķinot lidojuma attālumu dažādiem parametriem var atrast funkcionālu sakarību starp lidojuma attālumu, izsviešanas ātrumu, sākotnējo augstumu, berzes koeficientu un izsviešanas leņki: $${x_{att}}=x_{att}(v_0, h_0, \alpha, \gamma)$$ Apvēršot to pret $v_0$ var iegūt arī sakarību $$v_0=v_0(x_{att} , h_0, \alpha, \gamma)$$
Izmantojot šīs sakarības var atrast optimālo izsviešanas leņķi, zinot izsviešanas ātrumu (maksimizējot attālumu), vai arī sasniedzamo lidojuma attālumu (minimizējot izviešanas ātrumu).  
Attāluma izmaiņas, izmainot ātrumu par vienu vienību var noteikt izmantojot lidojuma attāluma atvasinājumu pēc izsviešanas ātruma - $\frac {\partial x_{att}}{\partial v_0}$

Nākamajā šūnā diferenciālvienādojumi izteikti kā funkcija, ko vēlāk lietosim diferenciālvienādojumu sistēmas atrisināšanas apakšprogrammā *scipy.integrate.odeint*. Šīs funckijas pirmais parametrs ir masīvs ar tik elementiem, cik ir mainīgo (mūsu gadījumā 4), otrais parametrs ir neatkarīgais mainīgais (mūsu gadījumā laiks $t$), tālākie parametri ir brīvie parametri (mūsu gadījumā $g$ un $\gamma$)

## Dotās aprēķina formulas

#### Definīcijas

<div class="alert alert-block alert-warning">
<b>Uzmanību!</b> Ieraksti šūnā zemāk pareizo funkciju, kas atbilst rēķināmajai DV sistēmai. Izmanto vienādojumos arī g un gamma!
</div>

In [19]:
def drdt(t, r, g, gamma):
    """
    Returns the derivatives for the system:
      dx/dt = v_x
      dz/dt = v_z
      dv_x/dt = - gamma * v_x * sqrt(v_x^2 + v_z^2)
      dv_z/dt = - g - gamma * v_z * sqrt(v_x^2 + v_z^2)
    
    Parameters
    ----------
    t : float
        Current time (not used explicitly in the equations, but required by ODE solvers).
    r : array_like
        Current values of [x, z, v_x, v_z].
    g : float
        Gravitational acceleration.
    gamma : float
        Drag coefficient (related to air resistance).
        
    Returns
    -------
    dydt : list
        The derivatives [dxdt, dzdt, dvxdt, dvzdt].
    """
    x, z, vx, vz = r
    speed = np.sqrt(vx**2 + vz**2)  # magnitude of the velocity vector

    dxdt = vx
    dzdt = vz
    dvxdt = -gamma * vx * speed
    dvzdt = -g - gamma * vz * speed

    dydt = [dxdt, dzdt, dvxdt, dvzdt]
    return dydt


In [20]:
import scipy.integrate
import scipy.optimize
import scipy.misc

#### Diferenciālvienādojuma risinātājs

In [21]:
def solve(tend, gamma, h0, v0, alpha):
    alpha_r=np.radians(alpha)
    r0=[0,h0,v0*np.cos(alpha_r),v0*np.sin(alpha_r)]
    tlist=np.linspace(0,tend,100) # saraksts ar sekundem, kur izvada atrisinājumu    
    tlist = np.squeeze(tlist)
    res=scipy.integrate.solve_ivp(drdt,  [min(tlist), max(tlist)],r0,t_eval =tlist,  args=(g, gamma))

    if not res.success:
        print(res)
        raise Exception('Error in solve')
    return tlist,res.y.T # Atgriež laika soļus, attālumu, augstumu, horizontālo ātrumu un vertikālo ātrumu

In [22]:
gamma = 0.15
tend = 10,
h0 = 2
v0 = 45 
alpha = 30
tlist, res = solve(tend, gamma, h0, v0,alpha)

### Šī funkcija automātiski aprēķina lidojuma attālumu

In [23]:
def xattalums(gamma, h0, v0, alpha):
    tguess=4.0 # Nejaušs minējums optimizācijas sākuma punktam
    max_t = 120 # maksimāli iespējamais lidojuma laiks (bisekcijas solverim)
    res=scipy.optimize.root_scalar(lambda t:solve(t, gamma, h0, v0, alpha)[1][-1,1], method='brentq', bracket=[1e-6,max_t])
    tlidojuma=res.root
    tlist,res=solve(tlidojuma, gamma, h0, v0, alpha) #Lidojuma attālums
    return res[-1,0]
# vektorizēta xattalums funkcija    
vxattalums = np.vectorize(xattalums)

#### Funkcionālsakarības apvēršana - aprēķinam izsviešanas ātrumu no lidojuma attāluma
Kā mainīgie: \
gamma - Berzes koeficients\
ho - Sākotnējais augstum\salpha - izmešanas leņķis\
xatt - Lidojuma attālumsms

In [24]:
def izmesanasatrums(gamma, h0, alpha, xatt):
    vguess=30.0 #Nejaušs minējums optimizācijas sākuma punktam
    vmin = 0.01 # minimālais ātrums bisekcijai
    vmax = 500 # maksimālais ātrums bisekcijai
    #return scipy.optimize.root(lambda v:vxattalums(gamma, h0, v, alpha)-xatt,vguess).x[0] #Lidojuma ātrums
    res = scipy.optimize.root_scalar(lambda v:xattalums(gamma, h0, v, alpha)-xatt,method='brentq', bracket=[vmin,vmax])   
    return res.root #Lidojuma ātrums
izmesanasatrums_vec=np.vectorize(izmesanasatrums) # Izmešanas ātruma aprēķina definīcija

#### Izsviešanas leņķa optimizācijas funkcijas (1) pie fiksēta v0 - *optimalais_lenkis_v* (2) pie fiksēta attāluma - *optimalais_lenkis_x*

In [25]:
def optimalais_lenkis_v(gamma, h0, v0):
    alpha_guess=45.0 #Nejaušs minējums optimizācijas sākuma punktam
    res=scipy.optimize.minimize(lambda alpha:-vxattalums(gamma,h0, v0, alpha), alpha_guess) # Optimālais leņķis pie dota ātruma
    return res.x[0]
def optimalais_lenkis_x(gamma, h0, xatt):
    alpha_guess=45.0 #Nejaušs minējums optimizācijas sākuma punktam
    alpha_min=1
    alpha_max=89
    res=scipy.optimize.minimize_scalar(lambda alpha:izmesanasatrums(gamma,h0, alpha, xatt), method='bounded', bounds=[alpha_min,alpha_max]) # Optimālais leņķis pie dota izmešanas leņķa
    return res.x

### Situācija bez pretestības  h0=2.0 m, attālums 90 m, optimālais leņkis

In [26]:
h0=2.0 #Izmešanas augstums 
xatt=90.0 #Nolidotais attālums
alphaopt_0=optimalais_lenkis_x(0.0, h0, xatt) #Optimālā leņķa aprēķins
dv=1.0 # Ātruma izmaiņa attāluma izmaiņas novērtēšanai
v0=izmesanasatrums(0.0,h0,alphaopt_0, xatt) #Izmešanas ātruma aprēķins
xprim=xattalums(0.0, h0, v0+dv, alphaopt_0)
print("Optimālais leņkis={alpha} v0={v0} m/s, deltaX={dx} m".format(alpha=alphaopt_0,v0=v0,dx=xprim-xatt))


Optimālais leņkis=44.36348496585332 v0=29.385336021442097 m/s, deltaX=6.094148331726188 m


### Situācija ar pretestību (v_terminal=50 m/s), h0=2.0 m, attālums 90 m, optimālais leņkis

In [27]:
h0=2.0 #Izmešanas augstums 
v_terminal=50 # Terminal velocity
gamma=g/(v_terminal**2)
xatt=90.0 #Nolidotais attālums
alphaopt_pret=optimalais_lenkis_x(gamma, h0, xatt)
dv=1.0 # atruma izmaina
v0=izmesanasatrums(gamma,h0,alphaopt_pret, xatt)
xprim=xattalums(gamma, h0, v0+dv, alphaopt_pret)
print("Optimālais leņkis={alpha} v0={v0} m/s, deltaX={dx} m".format(alpha=alphaopt_pret,v0=v0,dx=xprim-xatt))

Optimālais leņkis=42.10732804744377 v0=34.04231788471205 m/s, deltaX=3.9571432463779956 m


# Uzdevumi

## Pamatuzdevumi
Iepazīties ar funkcijām:
`xattalums`, `izmesanasatrums`, `optimalais_lenkis_v`, `optimalais_lenkis_x`, izskaidrot, ko katra funkcija dara un sniegt piemērus. 

Izskaidrojiet `lambda funkciju` lietojumu šajā uzdevumā.

## Papilduzdevumi:
1. Kāds būtu optimālais leņķis, lai jūs trāpītu mērķī, kas atrodas uz zemes 30m attālumā ar pēc iespējas mazāku izmešanas ātrumu? Un kāds būtu šis ātrums?
2. Cik lielam būtu jābūt mērķim, lai jūs droši trāpītu ar nelielu (deltav=2m/s) rokas trīcēšanu?
3. Cik tālā mērķī jūs varētu trāpīt ar pie optimālā leņķa? Izmantojiet savu augumu un novērtējiet savu sviešanas ātrumu!
   


In [28]:
# 1

xatt_30 = 30.0
h0 = 2.0
gamma = 0.0  # no drag for simplicity

# Use 'optimalais_lenkis_x' to find the angle that yields the smallest v0 for hitting 30m
alpha_opt_30 = optimalais_lenkis_x(gamma, h0, xatt_30)

# Now find the actual minimal v0 using 'izmesanasatrums'
v0_opt_30 = izmesanasatrums(gamma, h0, alpha_opt_30, xatt_30)

print("Papilduzdevums 1 (bez pretestības):")
print(f"  - Optimālais leņķis, lai trāpītu 30m ar minimālu ātrumu = {alpha_opt_30:.2f} grādi")
print(f"  - Minimālais izmešanas ātrums = {v0_opt_30:.2f} m/s")

Papilduzdevums 1 (bez pretestības):
  - Optimālais leņķis, lai trāpītu 30m ar minimālu ātrumu = 43.09 grādi
  - Minimālais izmešanas ātrums = 16.59 m/s


In [29]:
# 2

dv = 2.0  # Variation in throw speed
v0_low = v0_opt_30 - dv
v0_high = v0_opt_30 + dv

# Compute the new distances
x_low = xattalums(gamma, h0, v0_low, alpha_opt_30)
x_high = xattalums(gamma, h0, v0_high, alpha_opt_30)

# The required target width is the difference
target_size = x_high - x_low

print("\nPapilduzdevums 2 (bez pretestības):")
print(f"  - Izmešanas ātrums var svārstīties ±{dv} m/s ap {v0_opt_30:.2f} m/s")
print(f"  - Lidojuma attālums svārstās no {x_low:.2f} m līdz {x_high:.2f} m")
print(f"  - Tātad mērķa garumam jābūt vismaz {target_size:.2f} m")


Papilduzdevums 2 (bez pretestības):
  - Izmešanas ātrums var svārstīties ±2.0 m/s ap 16.59 m/s
  - Lidojuma attālums svārstās no 23.62 m līdz 37.18 m
  - Tātad mērķa garumam jābūt vismaz 13.56 m


In [30]:
# 3) Max distance with my personal throw speed estimate
my_estimated_v0 = 20.0  # for example, guess you can throw ~20 m/s
h0 = 1.85  # my height
gamma = 0.0  # no drag

alpha_opt_my = optimalais_lenkis_v(gamma, h0, my_estimated_v0)
range_my = xattalums(gamma, h0, my_estimated_v0, alpha_opt_my)

print("\nPapilduzdevums 3 (bez pretestības):")
print(f"  - Pieņemsim, ka varat mest ar ~{my_estimated_v0:.1f} m/s.")
print(f"  - Aprēķinātais optimālais leņķis = {alpha_opt_my:.2f}°")
print(f"  - Attālums, ko varētu sasniegt, = {range_my:.2f} m")


Papilduzdevums 3 (bez pretestības):
  - Pieņemsim, ka varat mest ar ~20.0 m/s.
  - Aprēķinātais optimālais leņķis = 43.76°
  - Attālums, ko varētu sasniegt, = 42.58 m
